In [ ]:
!pip install qdrant-client sentence-transformers numpy pandas tqdm

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv

load_dotenv()
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_COLLECTION_JOBS = os.getenv("QDRANT_COLLECTION_JOBS")

In [8]:
from tqdm.notebook import tqdm

In [19]:
conn = sqlite3.connect('convexhire.db')

In [20]:
pd.set_option('display.max_columns', None)

In [39]:
sql_query = "SELECT * FROM job;"
df = pd.read_sql_query(sql_query, conn)
df.head(10)

,job_id,title,department,level,description,location,location_type,is_remote,employment_type,salary_min,salary_max,salary_currency,requirements,skills,benefits,posted_date,application_deadline,status,is_featured,applicant_count,views_count,company_id,created_by,created_at,updated_at
0,1,Senior Full Stack Engineer,Engineering,Senior,Join our engineering team to build scalable cl...,"San Francisco, CA",Hybrid,0,Full-time,140000,180000,USD,"[""5+ years of full stack development experienc...","[""React"", ""Node.js"", ""Python"", ""AWS"", ""Kuberne...","[""Competitive salary and equity"", ""Health, den...",2025-10-14,2025-11-13,Active,1,0,0,1,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354237,2025-10-14 06:09:04.354244
1,2,Data Scientist,Data Science,Mid,Analyze complex datasets and build predictive ...,"New York, NY",Hybrid,0,Full-time,110000,145000,USD,"[""3+ years of data science experience"", ""Stron...","[""Python"", ""Machine Learning"", ""SQL"", ""TensorF...","[""Competitive compensation"", ""Comprehensive he...",2025-10-14,2025-11-08,Active,1,0,0,2,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354371,2025-10-14 06:09:04.354375
2,3,DevOps Engineer,Infrastructure,Mid,"Build and maintain CI/CD pipelines, manage clo...",Remote,Remote,1,Full-time,120000,155000,USD,"[""3+ years of DevOps/SRE experience"", ""Strong ...","[""AWS"", ""Kubernetes"", ""Docker"", ""Terraform"", ""...","[""Fully remote position"", ""Competitive salary ...",2025-10-14,2025-11-18,Active,1,0,0,3,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354433,2025-10-14 06:09:04.354436
3,4,Backend Engineer,Engineering,Junior,Develop and maintain backend services for our ...,"Austin, TX",On-site,0,Full-time,85000,110000,USD,"[""1-2 years of backend development experience""...","[""Java"", ""Python"", ""PostgreSQL"", ""REST APIs"", ...","[""Mentorship from senior engineers"", ""Health i...",2025-10-14,2025-11-23,Active,0,0,0,4,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354488,2025-10-14 06:09:04.354491
4,5,React Native Developer,Mobile,Mid,"Build beautiful, performant mobile application...","Los Angeles, CA",Hybrid,0,Full-time,115000,145000,USD,"[""3+ years of React Native development"", ""Stro...","[""React Native"", ""TypeScript"", ""Redux"", ""iOS"",...","[""Competitive salary"", ""Hybrid work schedule"",...",2025-10-14,2025-11-11,Active,0,0,0,5,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354539,2025-10-14 06:09:04.354543
5,6,Frontend Engineer,Engineering,Mid,Create exceptional user experiences with moder...,Remote,Remote,1,Full-time,105000,135000,USD,"[""3+ years of frontend development experience""...","[""React"", ""TypeScript"", ""CSS"", ""Tailwind"", ""Ne...","[""Fully remote"", ""Flexible hours"", ""Health ins...",2025-10-14,2025-11-15,Active,1,0,0,1,dcbab282-9198-4f48-a6da-dbad763bf052,2025-10-14 06:09:04.354589,2025-10-14 06:09:04.354592


In [22]:
[row.title + "," + row.description + "," + row.skills for row in df.itertuples()]

['Senior Full Stack Engineer,Join our engineering team to build scalable cloud infrastructure. Work on cutting-edge technologies including Kubernetes, microservices, and serverless architectures.,["React", "Node.js", "Python", "AWS", "Kubernetes", "TypeScript"]',
 'Data Scientist,Analyze complex datasets and build predictive models to drive business insights. Work with cutting-edge ML frameworks and big data technologies.,["Python", "Machine Learning", "SQL", "TensorFlow", "pandas", "Tableau"]',
 'DevOps Engineer,Build and maintain CI/CD pipelines, manage cloud infrastructure, and ensure system reliability. Work with modern DevOps tools and practices.,["AWS", "Kubernetes", "Docker", "Terraform", "Jenkins", "Python"]',
 'Backend Engineer,Develop and maintain backend services for our payment processing platform. Learn from experienced engineers and grow your skills.,["Java", "Python", "PostgreSQL", "REST APIs", "Git", "Linux"]',
 'React Native Developer,Build beautiful, performant mobile

In [23]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cuda"
)

In [24]:
texts = [
    f"{row.title}. {row.description}. {row.skills}"
    for row in df.itertuples(index=False)
]
texts

['Senior Full Stack Engineer. Join our engineering team to build scalable cloud infrastructure. Work on cutting-edge technologies including Kubernetes, microservices, and serverless architectures.. ["React", "Node.js", "Python", "AWS", "Kubernetes", "TypeScript"]',
 'Data Scientist. Analyze complex datasets and build predictive models to drive business insights. Work with cutting-edge ML frameworks and big data technologies.. ["Python", "Machine Learning", "SQL", "TensorFlow", "pandas", "Tableau"]',
 'DevOps Engineer. Build and maintain CI/CD pipelines, manage cloud infrastructure, and ensure system reliability. Work with modern DevOps tools and practices.. ["AWS", "Kubernetes", "Docker", "Terraform", "Jenkins", "Python"]',
 'Backend Engineer. Develop and maintain backend services for our payment processing platform. Learn from experienced engineers and grow your skills.. ["Java", "Python", "PostgreSQL", "REST APIs", "Git", "Linux"]',
 'React Native Developer. Build beautiful, performa

In [25]:
vectors = model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
vectors

array([[-0.02416821, -0.02880373,  0.08208416, ..., -0.03736752,
        -0.00331066,  0.00910225],
       [-0.00997357, -0.06150351, -0.00562632, ..., -0.06321636,
        -0.0137971 ,  0.03574916],
       [ 0.002434  ,  0.00060705,  0.0636327 , ..., -0.02245474,
         0.03088372,  0.01583491],
       [-0.03367468, -0.03481144, -0.01257724, ...,  0.0036023 ,
         0.06078191,  0.00811241],
       [-0.05593196, -0.0410427 ,  0.12492653, ...,  0.0017355 ,
         0.05007673,  0.0465199 ],
       [-0.05071673, -0.02394487,  0.02112311, ...,  0.00856446,
         0.00063305,  0.08031742]], dtype=float32)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

In [ ]:
collection_name = QDRANT_COLLECTION_JOBS

if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=model.get_sentence_embedding_dimension(),
            distance=Distance.COSINE,
        ),
    )

In [52]:
payloads = [
    {
        "job_id": row.job_id,
        "title": row.title,
        "department": row.department,
        "level": row.level,
        "location": row.location,
        "location_type": row.location_type,
        "employment_type": row.employment_type,
        "salary_min": row.salary_min,
        "salary_max": row.salary_max,
        "salary_currency": row.salary_currency,
        "company_id": row.company_id,
        "posted_date": row.posted_date,
        "status": row.status,
    }
    for row in df.itertuples(index=False)
]

In [53]:
client.upload_collection(
    collection_name=collection_name,
    vectors=vectors,
    payload=payloads,
    ids=None,
    batch_size=128,
)

print("Uploaded vectors with title metadata successfully!")

Uploaded vectors with title metadata successfully!


In [58]:
for i, hit in enumerate(search_results.points):
    title = hit.payload.get("title", "N/A")
    level = hit.payload.get("level", "N/A")
    company_id = hit.payload.get("company_id", "N/A")
    score = hit.score

    print(f"{i+1}. {title}  (score={score:.3f})   Level: {level}   Company ID: {company_id}")

1. Data Scientist  (score=0.170)   Level: Mid   Company ID: 2
2. Senior Full Stack Engineer  (score=0.100)   Level: Senior   Company ID: 1
3. Backend Engineer  (score=0.094)   Level: Junior   Company ID: 4
4. Frontend Engineer  (score=0.082)   Level: Mid   Company ID: 1
5. React Native Developer  (score=0.070)   Level: Mid   Company ID: 5


In [ ]:
# client.delete_collection(collection_name=QDRANT_COLLECTION_JOBS)

True

In [64]:
for i in search_results.points:
  print(i)

id='e6ee07d9-88af-40b9-87d9-62d017c95de4' version=0 score=0.16968471 payload={'job_id': 2, 'title': 'Data Scientist', 'department': 'Data Science', 'level': 'Mid', 'location': 'New York, NY', 'location_type': 'Hybrid', 'employment_type': 'Full-time', 'salary_min': 110000, 'salary_max': 145000, 'salary_currency': 'USD', 'company_id': 2, 'posted_date': '2025-10-14', 'status': 'Active'} vector=None shard_key=None order_value=None
id='2b9fface-11d5-496a-8d80-6b4cf000a709' version=0 score=0.10018246 payload={'job_id': 1, 'title': 'Senior Full Stack Engineer', 'department': 'Engineering', 'level': 'Senior', 'location': 'San Francisco, CA', 'location_type': 'Hybrid', 'employment_type': 'Full-time', 'salary_min': 140000, 'salary_max': 180000, 'salary_currency': 'USD', 'company_id': 1, 'posted_date': '2025-10-14', 'status': 'Active'} vector=None shard_key=None order_value=None
id='5af5f7d5-e415-4e03-9b79-337642902e11' version=0 score=0.09371268 payload={'job_id': 4, 'title': 'Backend Engineer',